## 환경 맞추기

In [2]:
!wget https://www.python.org/ftp/python/3.7.0/Python-3.7.0.tgz
!tar xvfz Python-3.7.0.tgz
!Python-3.7.0/configure
!make
!sudo make install

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_calltips.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_codecontext.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_colorizer.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_config.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_config_key.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_configdialog.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_debugger.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_delegator.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_editmenu.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_editor.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_grep.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_help.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_help_a

In [3]:
!python --version

Python 3.7.0


In [4]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

  Using cached mxnet-1.9.1-py3-none-manylinux2014_x86_64.whl (49.1 MB)
  Using cached graphviz-0.8.4-py2.py3-none-any.whl (16 kB)
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
  Using cached gluonnlp-0.10.0.tar.gz (344 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661749 sha256=50bdf95f99f3a3255fdfacfbcbaac63032876d5115b9e96fcc9d935b9dff3357
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-

In [5]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-e87gq5gn/kobert-tokenizer_356bd8096fe447c0b1609a9ecbc2f40f
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-e87gq5gn/kobert-tokenizer_356bd8096fe447c0b1609a9ecbc2f40f
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=3f9a46c04bde6d341ac368f2a214090e5858e8e7ca4a43e9d1732fd146975a69
  Stored in directory: /tmp/pip-ephem-wheel-cache-mv7qsl_n/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [6]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.1 MB/s

## 라이브러리 임포트

In [7]:
import pandas as pd
import numpy as np
# np.bool가 bool로 대체되었음을 명시
np.bool = bool

In [8]:
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
from tensorflow.keras.preprocessing.text import Tokenizer

In [9]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup
from sklearn.metrics.pairwise import cosine_similarity

from transformers import TrainingArguments, Trainer


## 데이터 처리

In [11]:
data= pd.read_csv('/content/ChatbotData.csv')
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


### (사전)EDA

0: 일반, 1: 사랑, 2: 이별

In [21]:
data['label'].value_counts()

label
0    5290
1    3570
2    2963
Name: count, dtype: int64

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


결측치 x

In [23]:
len(max(data['Q'], key= len))

56

### 데이터 전처리

In [12]:
love_data= data[data['label'] != 0]
love_data

,Q,A,label
5290,1000일 만난 여자친구와 이별,더 오래 만날 사람 만날 거예요.,1
5291,10년 연애. 헤어졌습니다.,더 공허함이 크시겠네요.,1
5292,10년 연애사 되돌아보니 다 부질없네,더 좋은 사람 만나실 거예요.,1
5293,10년 연예의끝,더 마음이 허하겠어요.,1
5294,10년만나다 헤어지네,충분히 슬퍼하고 충분히 아파하다가 이겨내세요.,1
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [13]:
from sklearn.model_selection import train_test_split

x= list(love_data['Q'])
y= list(love_data['A'])
label= list(love_data['label'])

# x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.2, random_state= 16)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

### (사전)사용자 정의 사전 추가 및 저장

In [27]:
token= Tokenizer()
token.fit_on_texts(x)
x_token= [word for word, _ in token.word_index.items()]
x_token

['좋아하는',
 '너무',
 '썸',
 '이별',
 '사람',
 '내가',
 '사람이',
 '거',
 '어떻게',
 '같아',
 '헤어진지',
 '내',
 '것',
 '나',
 '싶어',
 '이제',
 '다시',
 '정말',
 '여자친구가',
 '수',
 '좀',
 '안',
 '남자친구가',
 '더',
 '잘',
 '오늘',
 '나를',
 '왜',
 '있을까',
 '자꾸',
 '또',
 '연애',
 '힘드네',
 '해',
 '있어',
 '될까',
 '타는',
 '있는데',
 '짝남이',
 '그',
 '하는',
 '연락',
 '짝사랑',
 '진짜',
 '사랑',
 '있는',
 '다',
 '날',
 '헤어진',
 '짝남',
 '후',
 '많이',
 '할까',
 '계속',
 '방법',
 '짝녀가',
 '다른',
 '이',
 '사랑이',
 '참',
 '모르겠어',
 '결국',
 '연락이',
 '없어',
 '사랑하는',
 '없는',
 '나한테',
 '마음이',
 '그냥',
 '전',
 '남자',
 '여자가',
 '여자친구',
 '남자가',
 '이젠',
 '힘들어',
 '같이',
 '결혼',
 '짝남한테',
 '헤어지고',
 '아직도',
 '먼저',
 '이렇게',
 '하고',
 '돼',
 '혼자',
 '여자',
 '어떤',
 '어제',
 '알았는데',
 '한',
 '줄',
 '못',
 '좋아',
 '이별을',
 '무슨',
 '때',
 '같은데',
 '안돼',
 '마음',
 '나는',
 '보고',
 '좋은',
 '했는데',
 '한달',
 '여친',
 '좋을까',
 '사람이랑',
 '이별한지',
 '싶다',
 '카톡',
 '나만',
 '해도',
 '갑자기',
 '마지막',
 '좋겠어',
 '사람을',
 '사람한테',
 '짝녀',
 '시간이',
 '지금',
 '만나고',
 '힘든',
 '걸까',
 '남자친구',
 '싶은데',
 '헤어졌습니다',
 '같은',
 '어떡해',
 '여친이',
 '이별후',
 '제가',
 '때문에',
 '합니다',
 '말',
 '오늘도',
 '걸',

In [28]:
# 사전학습 토크나이저와 모델 불러오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1')#, return_dict=False) # 튜플 반환

# 사용자 정의 토크나이저 사용
## 사용자 정의 단어, 토크나이저에 추가
tokenizer.add_tokens(x_token)
## 모델의 임베딩 층 확장
model.resize_token_embeddings(len(tokenizer))

model.to(device)

# 추가된 상태의 모델과 토크나이저 저장
tokenizer.save_pretrained('./tokenizer_with_custom_vocab')
model.save_pretrained('./model_with_custom_vocab')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

### (사전) 질문을 임베딩 벡터로 변환하여 저장

In [29]:
# 저장된 모델 및 토크나이저 로드
tokenizer = KoBERTTokenizer.from_pretrained('/content/drive/MyDrive/groom/tokenizer_with_custom_vocab')
model = BertModel.from_pretrained('/content/drive/MyDrive/groom/model_with_custom_vocab')
model.to(device)
model.eval()

# 질문을 임베딩 벡터로 변환하는 함수
def embed_question(question):
    with torch.no_grad():
        # 인덱스 (texts_to_sequence 작업과 유사)
        encoding = tokenizer.encode_plus(
            question,
            add_special_tokens=True,   # [CLS], [PAD] 등
            max_length=60,
            padding='max_length',      # 패딩
            truncation=True,           # 자르기
            return_attention_mask=True,# [PAD]는 0, 나머지는 1
            return_tensors='pt',       # pytorch tensor로 반환
        )
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)

        # 임베딩
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        cls_embedding = last_hidden_state[:, 0, :].cpu().numpy()
        return cls_embedding           # 문장의 전체의미를 요약하는 [CLS]토큰

# 모든 질문을 임베딩 벡터로 변환하여 저장
embedded_questions = np.vstack([embed_question(q) for q in x])
np.save('embedded_questions_kobert.npy', embedded_questions)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 유사도

In [15]:
# 모델과 토크나이저 로드
tokenizer = KoBERTTokenizer.from_pretrained('/content/drive/MyDrive/groom/tokenizer_with_custom_vocab')
model = BertModel.from_pretrained('/content/drive/MyDrive/groom/model_with_custom_vocab')
model.to(device)
model.eval()

# 임베딩 벡터로 변환
def get_embedding(question):
    with torch.no_grad():
        # 인덱스 (texts_to_sequence 작업과 유사)
        encoding = tokenizer.encode_plus(
            question,
            add_special_tokens=True,   # [CLS], [PAD] 등
            max_length=60,
            padding='max_length',      # 패딩
            truncation=True,           # 자르기
            return_attention_mask=True,# [PAD]는 0, 나머지는 1
            return_tensors='pt',       # pytorch tensor로 반환
        )
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)

        # 임베딩
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        cls_embedding = last_hidden_state[:, 0, :].cpu().numpy()
        return cls_embedding           # 문장의 전체의미를 요약하는 [CLS]토큰

# 기존 questions의 임베딩 값
question_embeddings = np.load('/content/drive/MyDrive/groom/embedded_questions_kobert.npy')

def find_best_answer(user_input):
    user_embedding = np.vstack(get_embedding(user_input))

    # 코사인 유사도 방식을 통해 유사 질문 데이터 찾기
    similarities = [cosine_similarity(user_embedding, q.reshape(1,768))[0][0] for q in question_embeddings]

    # 가장 유사도가 높은 질문을 찾아 그에 대응하는 답변 뱉기
    max_similarity = max(similarities)
    print('max_similarity', max_similarity)
    if max_similarity > 0.8:  # 임계값을 통해 무관한 질문 필터링
        best_match_index = similarities.index(max_similarity)
        return y[best_match_index]
    else:
        return "저는 연애 관련 상담만을 주로 하고 있어요. 다른 질문 부탁해요!"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
# 중복있나?
print(len(x))
print(len(set(x)))

6533
6478


In [17]:
question_embeddings[0].shape

(768,)

In [18]:
# 사용
print("저는 당신의 연애/사랑/이별 에 대한 고민에 대해 하나의 답을 내려줄 챗봇 상담가 '러브레터'입니다 🙂✉️🩷")
while True:
    user_question = input("\n요즘 당신의 고민은 뭔가요? ('종료' 입력시 채팅이 종료됩니다.)\n   >>  ")
    if user_question == '종료':
        print("행운을 빌어요🍀")
        break
    print(find_best_answer(user_question))

저는 당신의 연애/사랑/이별 에 대한 고민에 대해 하나의 답을 내려줄 챗봇 상담가 '러브레터'입니다 🙂✉️🩷

요즘 당신의 고민은 뭔가요? ('종료' 입력시 채팅이 종료됩니다.)
   >>  외로워
max_similarity 0.99999994
제가 곁에 있을게요.

요즘 당신의 고민은 뭔가요? ('종료' 입력시 채팅이 종료됩니다.)
   >>  종료
행운을 빌어요🍀


## 그라디오 UI

In [19]:
import gradio as gr

# 챗봇에 채팅이 입력되면 이 함수를 호출합니다.
# message는 유저의 채팅 메시지, history는 채팅 기록, additional_input_info는 additional_inputs안 블록의 정보를 받습니다.
def response(message, history, additional_input_info):

    best_answer = find_best_answer(message)
    # additional_input_info의 텍스트를 챗봇의 대답 뒤에 추가합니다.
    return best_answer

    # # additional_input_info의 텍스트를 챗봇의 대답 뒤에 추가합니다.
    # return "챗봇을 미완성하였습니다 " + additional_input_info

gr.ChatInterface(
        fn=response,
        textbox=gr.Textbox(placeholder="어떤 연애/사랑/이별 고민이 있나요?", container=False, scale=7),
        title="🩷🪄Love Lecture🪄🩷",
        description="저는  당신의  연애/사랑/이별에  대한  고민에  하나의  답을  내려줄  챗봇  상담가  '러브레터'입니다 🙂🩷",
        theme="soft",
        examples=[["좋아하는 사람이랑 친구 가능해?"], ["이별이 너무 고통스러워"], ["결혼식 앞두고 고민이야"]],
        retry_btn="다시보내기 ↩",
        undo_btn="이전챗 삭제 ❌",
        clear_btn="전챗 삭제 💫",
        additional_inputs=[
            gr.Textbox("!!!", label="끝말잇기")
        ]
).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9334d98581c52971bb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## 모델 파인 튜닝

In [42]:
# 예시 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = tokenizer(text, padding='max_length', truncation=True, max_length=128)  # 토크나이징

        input_ids = torch.tensor(encoding['input_ids'], dtype=torch.long)
        attention_mask = torch.tensor(encoding['attention_mask'], dtype=torch.long)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [43]:
max_len = 64
batch_size = 32
num_epochs = 5
learning_rate =  5e-5

In [34]:
len(label)

6533

In [44]:
data_train = CustomDataset(x[:6000], label[:6000])
data_test = CustomDataset(x[6000:], label[6000:])

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [50]:
# 옵티마이저와 학습 스케줄러 설정

# AdamW 는 Adam 옵티마이저의 변형으로, L2 정규화 (weight decay)를 더 잘 구현함
optimizer = AdamW(model.parameters(), lr= learning_rate, correct_bias=False)
total_steps = len(train_dataloader) * num_epochs  # epochs

# 학습 초기에 학습률을 천천히 증가(warmup)시키고 이후에는 선형적으로 감소 > 안정적 학습을 위해
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [45]:
# 손실 함수 설정
loss_fn = torch.nn.CrossEntropyLoss()

# 모델 학습 함수 정의
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler):
    model.train()
    losses = []
    correct_predictions = 0

    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)


In [46]:
# 모델 평가 함수 정의
def eval_model(model, data_loader, loss_fn, device):
    model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)


In [52]:
# 학습 과정

for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{epoch}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(model, train_dataloader, loss_fn, optimizer, device, scheduler)
    print(f'Train loss {train_loss} accuracy {train_acc}')

    # val_acc, val_loss = eval_model(model, val_loader, loss_fn, device)
    # print(f'Val   loss {val_loss} accuracy {val_acc}')

# 학습된 모델 저장
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

Epoch 1/0
----------


TypeError: BertModel.forward() got an unexpected keyword argument 'labels'

optimizer와 schedule 설정

In [ ]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10,
    save_total_limit=2
)

In [ ]:
# 모델 저장
torch.save(model.state_dict(), 'model.pth')

In [ ]:
# 질문에 대한 응답 생성 함수
def get_response(question, model, tokenizer, max_len):
    model.eval()
    with torch.no_grad():
        encoding = tokenizer.encode_plus(
            question,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        return last_hidden_state.cpu().numpy()

In [ ]:
# 예시 질문에 대한 응답 생성
question = "연애 고민이 있어요"
response = get_response(question, model, tokenizer, max_len)
print(response)

[[[ 0.01390528  0.01303202  0.06411833 ... -0.01632296 -0.0916929
    0.06495484]
  [ 0.01472737  0.01168557  0.06451527 ... -0.01600198 -0.09181871
    0.06472722]
  [ 0.015371    0.01321049  0.06399427 ... -0.01560237 -0.09197576
    0.06562712]
  ...
  [ 0.0136602   0.01336958  0.0634004  ... -0.01581043 -0.09049875
    0.0654232 ]
  [ 0.01366229  0.01336842  0.06340727 ... -0.01581672 -0.09050184
    0.06541782]
  [ 0.01366462  0.01337132  0.06341007 ... -0.01581906 -0.0905064
    0.06541616]]]


---

## 모델 테스트

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from kobert_tokenizer import KoBERTTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 모델 및 토크나이저 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1')
model.load_state_dict(torch.load('/content/drive/MyDrive/groom/model.pth'))
model.to(device)

# 평가모드로 전환
model.eval()

# 데이터 로드
x = list(love_data['Q'])
y = list(love_data['A'])

# 질문을 임베딩 벡터로 변환하는 함수
def embed_question(question):
    with torch.no_grad():
        encoding = tokenizer.encode_plus(
            question,
            add_special_tokens=True,
            max_length=64,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        cls_embedding = last_hidden_state[:, 0, :].cpu().numpy()
        return cls_embedding

# 기존 질문들을 임베딩 벡터로 변환하여 저장
embedded_questions = np.vstack([embed_question(q) for q in x])

# 사용자 질문에 가장 유사한 질문을 찾아 답변을 반환하는 함수
def find_best_answer(user_question):
    user_embedding = embed_question(user_question)
    similarities = cosine_similarity(user_embedding, embedded_questions)
    best_match_idx = np.argmax(similarities)
    return y[best_match_idx]

# 챗봇 동작
print("저는 당신의 연애/사랑/이별에 대한 고민에 대해 하나의 답을 내려줄 챗봇 상담가 '러브레터'입니다 🙂✉️🩷")

while True:
    user_question = input("\n요즘 당신의 고민은 뭔가요? ('종료' 입력시 채팅이 종료됩니다.)\n   >>  ")
    if user_question.lower() == '종료':
        print("행운을 빌어요🍀")
        break
    answer = find_best_answer(user_question)
    print(answer)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


RuntimeError: Error(s) in loading state_dict for BertModel:
	size mismatch for embeddings.word_embeddings.weight: copying a param with shape torch.Size([16603, 768]) from checkpoint, the shape in current model is torch.Size([8002, 768]).

## references

https://velog.io/@sseq007/Kobert-%EB%AA%A8%EB%8D%B8-%EC%82%AC%EC%9A%A91

https://velog.io/@sseq007/Kobert-%EB%AA%A8%EB%8D%B8-%EC%82%AC%EC%9A%A92

https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab

https://complexoftaste.tistory.com/2

공식 : https://github.com/SKTBrain/KoBERT?tab=readme-ov-file#korean-bert-pre-trained-cased-kobert